### Exploration and Analysis of Shimmer IMU data of MP
In this notebook, we will perform the basic analysis of shimmer data. This data has been obtained from Shimer IMU placed on the different body parts of the participants. There are two sources of data: video data used to capture the execution and sensor data obtained from the sensors attached to the body of the participants.

In this notebook we currently consider the MP exercise. The raw data consists of a single csv file for each type of MP.

- Reading the data is quite easy. We just have to rename the file to .csv extension.
- Description 5 sensors (lower back, 2 wrists, 2 elbows)
- Accelerometer (movement acceleration)
        Calibrated and Uncalibrated
            We have to use the calibrated one
        Low noise and wide range
            These two are approximately the same. We only have to use the LN. 
        X , Y and Z coordinates
- Magnetometer (orientation/compass?)
        Calibrated and Uncalibrated
        X , Y and Z coordinates
- Gyroscope (angular velocity, degrees per second)
        Calibrated and Uncalibrated
        X , Y and Z coordinates
- Magnetometer can be used to do the segmentation
- Each sensor 33 columns
    12 from accelerometer
    6 from magnetometer
    6 from gyroscope
    Other columns
        Batt_percentage - 1
        Event Marker -1 
        Packer Reception Rate Trial
        Packet Reception Rate Current
        SyncTimestamp
        System Timestamp
            Cal and uncal
        Timestamp
            Cal and uncal
- We don’t have to use the uncal data, only cal data
- Total columns: 5 sensors, 5 * 33 columns + 2 extra columns (Annotation Level and Annotation Pulse)


In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
import sys

In [6]:
path = "/home/ashish/Results/Datasets/Shimmer/MP/RawSignal/"
cleaned_data_path = "/home/ashish/Results/Datasets/Shimmer/MP/CleanedSignal/"

In [7]:
list_of_files = os.listdir(path)

In [5]:
cols_each_file = [(i, pd.read_csv(os.path.join(path, i), sep="\t").shape[1]) for i in list_of_files]

In [36]:
kw = ["Batt_Percentage", "Packet_Reception_Rate_Current", "Packet_Reception_Rate_Trial", "Timestamp", "Level", "Pulse", "_WR", "Event_Marker"]

In [55]:
d_mapping = {"pid":[], "col_names": []}
for i in list_of_files:
    df = pd.read_csv(os.path.join(path, i), sep="\t")
    df.dropna(how='all', axis=1, inplace=True)
    uncal_columns = []
    unn_columns = []
    for c in df.columns:
        second_row = df.loc[1, c]
        first_row = df.loc[0, c]
        if type(second_row) == str and  "UNCAL" in second_row:
            uncal_columns.append(c)
        else:
            for k in kw:
                if type(first_row) == str and k in first_row:
                    unn_columns.append(c)
    df.drop(uncal_columns + unn_columns, axis=1, inplace=True)
    pid = i[:-4].split(" ")[0]
    exercise_type = i[:-4].split(" ")[-1]
    d_mapping["pid"].append("{}_{}".format(pid, exercise_type))
    
    # print(i)
    df = df.drop([1,2])
    new_column_names = []
    for c in df.columns:
        imu_name = c.split(".")[0]
        first_row = df.loc[0, c]
        combined_name = imu_name + "_" + first_row
        new_column_names.append(combined_name)
    df.columns = new_column_names
    df = df.drop([0])
    d_mapping["col_names"].append(list(df.columns))
    df.to_csv(os.path.join(cleaned_data_path, i), index=False)
    # print(df.shape)
    # print(df.columns)
    

In [35]:
df.head()

,Shimmer_B8C7,Shimmer_B8C7.2,Shimmer_B8C7.4,Shimmer_B8C7.13,Shimmer_B8C7.14,Shimmer_B8C7.16,Shimmer_B8C7.18,Shimmer_B8C7.20,Shimmer_B8C7.22,Shimmer_B8C7.24,...,S57D0Waist,S57D0Waist.2,S57D0Waist.4,S57D0Waist.13,S57D0Waist.14,S57D0Waist.16,S57D0Waist.18,S57D0Waist.20,S57D0Waist.22,S57D0Waist.24
0,Accel_LN_X,Accel_LN_Y,Accel_LN_Z,Event_Marker,Gyro_X,Gyro_Y,Gyro_Z,Mag_X,Mag_Y,Mag_Z,...,Accel_LN_X,Accel_LN_Y,Accel_LN_Z,Event_Marker,Gyro_X,Gyro_Y,Gyro_Z,Mag_X,Mag_Y,Mag_Z
1,CAL,CAL,CAL,CAL,CAL,CAL,CAL,CAL,CAL,CAL,...,CAL,CAL,CAL,CAL,CAL,CAL,CAL,CAL,CAL,CAL
2,m/(s^2),m/(s^2),m/(s^2),no_units,deg/s,deg/s,deg/s,local_flux,local_flux,local_flux,...,m/(s^2),m/(s^2),m/(s^2),no_units,deg/s,deg/s,deg/s,local_flux,local_flux,local_flux
3,-0.6987951807228916,7.903614457831326,5.9397590361445785,-1.0,-2.396946564885496,3.0839694656488548,5.633587786259541,-0.35454545454545455,-0.8254545454545454,0.5653061224489796,...,-0.8795180722891567,-5.46987951807229,7.83132530120482,-1.0,-3.66412213740458,2.763358778625954,-1.0229007633587786,0.14,0.20909090909090908,-0.6755102040816328
4,-0.42168674698795183,8.012048192771084,6.096385542168675,-1.0,-1.6335877862595418,2.595419847328244,3.282442748091603,-0.3563636363636364,-0.83,0.5602040816326531,...,-0.9036144578313253,-5.566265060240964,7.879518072289157,-1.0,-4.977099236641221,4.961832061068702,-0.9923664122137404,0.14545454545454545,0.22363636363636363,-0.6693877551020408


In [50]:
mapping_df = pd.DataFrame(d_mapping["col_names"])

In [51]:
mapping_df.head()

,0,1,2,3,4,5,6,7,8,9,...,35,36,37,38,39,40,41,42,43,44
0,Shimmer_B8C7_Accel_LN_X,Shimmer_B8C7_Accel_LN_Y,Shimmer_B8C7_Accel_LN_Z,Shimmer_B8C7_Gyro_X,Shimmer_B8C7_Gyro_Y,Shimmer_B8C7_Gyro_Z,Shimmer_B8C7_Mag_X,Shimmer_B8C7_Mag_Y,Shimmer_B8C7_Mag_Z,B8E3_Waist_Accel_LN_X,...,S548D_Left_Mag_Z,S57D0Waist_Accel_LN_X,S57D0Waist_Accel_LN_Y,S57D0Waist_Accel_LN_Z,S57D0Waist_Gyro_X,S57D0Waist_Gyro_Y,S57D0Waist_Gyro_Z,S57D0Waist_Mag_X,S57D0Waist_Mag_Y,S57D0Waist_Mag_Z
1,Shimmer_B8C7_Accel_LN_X,Shimmer_B8C7_Accel_LN_Y,Shimmer_B8C7_Accel_LN_Z,Shimmer_B8C7_Gyro_X,Shimmer_B8C7_Gyro_Y,Shimmer_B8C7_Gyro_Z,Shimmer_B8C7_Mag_X,Shimmer_B8C7_Mag_Y,Shimmer_B8C7_Mag_Z,B8E3_Waist_Accel_LN_X,...,S548D_Left_Mag_Z,S57D0Waist_Accel_LN_X,S57D0Waist_Accel_LN_Y,S57D0Waist_Accel_LN_Z,S57D0Waist_Gyro_X,S57D0Waist_Gyro_Y,S57D0Waist_Gyro_Z,S57D0Waist_Mag_X,S57D0Waist_Mag_Y,S57D0Waist_Mag_Z
2,Shimmer_B8C7_Accel_LN_X,Shimmer_B8C7_Accel_LN_Y,Shimmer_B8C7_Accel_LN_Z,Shimmer_B8C7_Gyro_X,Shimmer_B8C7_Gyro_Y,Shimmer_B8C7_Gyro_Z,Shimmer_B8C7_Mag_X,Shimmer_B8C7_Mag_Y,Shimmer_B8C7_Mag_Z,B8E3_Waist_Accel_LN_X,...,S548D_Left_Mag_Z,S57D0Waist_Accel_LN_X,S57D0Waist_Accel_LN_Y,S57D0Waist_Accel_LN_Z,S57D0Waist_Gyro_X,S57D0Waist_Gyro_Y,S57D0Waist_Gyro_Z,S57D0Waist_Mag_X,S57D0Waist_Mag_Y,S57D0Waist_Mag_Z
3,Shimmer_B8C7_Accel_LN_X,Shimmer_B8C7_Accel_LN_Y,Shimmer_B8C7_Accel_LN_Z,Shimmer_B8C7_Gyro_X,Shimmer_B8C7_Gyro_Y,Shimmer_B8C7_Gyro_Z,Shimmer_B8C7_Mag_X,Shimmer_B8C7_Mag_Y,Shimmer_B8C7_Mag_Z,B8E3_Waist_Accel_LN_X,...,S548D_Left_Mag_Z,S57D0Waist_Accel_LN_X,S57D0Waist_Accel_LN_Y,S57D0Waist_Accel_LN_Z,S57D0Waist_Gyro_X,S57D0Waist_Gyro_Y,S57D0Waist_Gyro_Z,S57D0Waist_Mag_X,S57D0Waist_Mag_Y,S57D0Waist_Mag_Z
4,Shimmer_B8C7_Accel_LN_X,Shimmer_B8C7_Accel_LN_Y,Shimmer_B8C7_Accel_LN_Z,Shimmer_B8C7_Gyro_X,Shimmer_B8C7_Gyro_Y,Shimmer_B8C7_Gyro_Z,Shimmer_B8C7_Mag_X,Shimmer_B8C7_Mag_Y,Shimmer_B8C7_Mag_Z,B8E3_Waist_Accel_LN_X,...,S548D_Left_Mag_Z,S57D0Waist_Accel_LN_X,S57D0Waist_Accel_LN_Y,S57D0Waist_Accel_LN_Z,S57D0Waist_Gyro_X,S57D0Waist_Gyro_Y,S57D0Waist_Gyro_Z,S57D0Waist_Mag_X,S57D0Waist_Mag_Y,S57D0Waist_Mag_Z


In [23]:
mapping_df["pid"] = d_mapping["pid"]

In [46]:
mapping_df.to_csv("./mapping_cols.csv", index=False, header=None)